<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

# ConvNets for CIFAR10 with TensorFlow and Keras <a class="tocSkip">

In [1]:
import os

import numpy as np
np.random.seed(123)
print("NumPy:{}".format(np.__version__))

import tensorflow as tf
tf.set_random_seed(123)
print("TensorFlow:{}".format(tf.__version__))

NumPy:1.13.1
TensorFlow:1.4.1


In [2]:
DATASETSLIB_HOME = '../datasetslib'
import sys
if not DATASETSLIB_HOME in sys.path:
    sys.path.append(DATASETSLIB_HOME)
%reload_ext autoreload
%autoreload 2
import datasetslib

datasetslib.datasets_root = os.path.join(os.path.expanduser('~'),'datasets')

# Get the CIFAR10 Data

In [3]:
from datasetslib.cifar import cifar10
from datasetslib import imutil
dataset = cifar10()
dataset.x_layout=imutil.LAYOUT_NHWC
dataset.load_data()
dataset.scaleX()

Already exists: /home/armando/datasets/cifar10/cifar-10-python.tar.gz
Extracting  /home/armando/datasets/cifar10/cifar-10-python.tar.gz


# CNN with TensorFlow for CIFAR10 Data

In [4]:
tf.reset_default_graph()

In [5]:
# define the hyperparameters
n_filters=[32,64]
learning_rate = 0.001

# input images of shape = (n_samples,n_width,n_height,n_depth)
x = tf.placeholder(dtype=tf.float32, name="x", 
                   shape=[None, dataset.width, dataset.height, dataset.depth]
                  ) 
# target output placeholder
y = tf.placeholder(dtype=tf.float32, name="y", shape=[None,dataset.n_classes]) 

keep_prob = tf.placeholder(tf.float32)

# create convolutional and pooling layers

# convolutional layer 1
layer1_w = tf.Variable(tf.random_normal(shape=[4,4,dataset.depth,n_filters[0]],
                                        stddev=0.01),
                       name='l1_w'
                      )
layer1_b = tf.Variable(tf.random_normal([n_filters[0]]),name='l1_b')
layer1_conv = tf.nn.relu(tf.nn.conv2d(x,
                                      layer1_w,
                                      strides=[1,1,1,1],
                                      padding='SAME'
                                     ) + 
                         layer1_b 
                        )
# pooling layer 1
layer1_pool = tf.nn.max_pool(layer1_conv,
                             ksize=[1,2,2,1],
                             strides=[1,2,2,1],
                             padding='SAME')
layer1_pool = tf.nn.dropout(layer1_pool, keep_prob)

# convolutional layer 2
layer2_w = tf.Variable(tf.random_normal(shape=[4,4,32,n_filters[1]],
                                        stddev=0.01),
                       name='l2_w'
                      )
layer2_b = tf.Variable(tf.random_normal([n_filters[1]]),
                       name='l2_b'
                      )
layer2_conv = tf.nn.relu(tf.nn.conv2d(layer1_pool,
                                      layer2_w,
                                      strides=[1,1,1,1],
                                      padding='SAME'
                                     ) + 
                         layer2_b 
                        )
# pooling layer 2
layer2_pool = tf.nn.max_pool(layer2_conv,
                             ksize=[1,2,2,1],
                             strides=[1,2,2,1],
                             padding='SAME')
layer2_pool = tf.nn.dropout(layer2_pool, keep_prob)

# fully connected layer
layer3_w = tf.Variable(tf.random_normal(shape=[8*8*64,1024],
                                        stddev=0.01),
                       name='l3_w')
layer3_b = tf.Variable(tf.random_normal([1024]),name='l3_b')
layer3_fc = tf.nn.relu(tf.matmul(tf.reshape(layer2_pool,[-1, 8*8*64]),
                                 layer3_w) +
                       layer3_b
                      )
layer3_fc = tf.nn.dropout(layer3_fc, keep_prob)

# output layer
layer4_w = tf.Variable(tf.random_normal(shape=[1024,dataset.n_classes],
                                        stddev=0.01),
                       name='l4_w')
layer4_b = tf.Variable(tf.random_normal([dataset.n_classes]),name='l4_b')

layer4_out = tf.matmul(layer3_fc,layer4_w)+layer4_b

model = layer4_out

# loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(logits=model, 
                                                  labels=y
                                                 )
loss = tf.reduce_mean(entropy)

# optimizer function
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(loss)

In [6]:
n_epochs = 20
dataset.y_onehot = True
dataset.batch_size = 128
dataset.batch_shuffle = True
n_batches = int(dataset.n_train/dataset.batch_size)

with tf.Session() as tfs:
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        total_loss = 0.0
        for batch in range(n_batches):
            batch_x,batch_y = dataset.next_batch()
            feed_dict={x:batch_x, y: batch_y, keep_prob: 1.0}
            batch_loss,_ = tfs.run([loss,optimizer],feed_dict=feed_dict)
            total_loss += batch_loss  
        average_loss = total_loss / n_batches
        print("Epoch: {0:04d}   loss = {1:0.6f}".format(epoch,average_loss))
    print("Model Trained.")

    predictions_check = tf.equal(tf.argmax(model,1),tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(predictions_check, tf.float32))
    feed_dict = {x:dataset.X_test, y:dataset.Y_test, keep_prob: 1.0}
    print("Accuracy:", accuracy.eval(feed_dict=feed_dict))

Epoch: 0000   loss = 2.187322
Epoch: 0001   loss = 1.823086
Epoch: 0002   loss = 1.605821
Epoch: 0003   loss = 1.477520
Epoch: 0004   loss = 1.360107
Epoch: 0005   loss = 1.255211
Epoch: 0006   loss = 1.152037
Epoch: 0007   loss = 1.051977
Epoch: 0008   loss = 0.951648
Epoch: 0009   loss = 0.861205
Epoch: 0010   loss = 0.767075
Epoch: 0011   loss = 0.679820
Epoch: 0012   loss = 0.601895
Epoch: 0013   loss = 0.517342
Epoch: 0014   loss = 0.443260
Epoch: 0015   loss = 0.378645
Epoch: 0016   loss = 0.319791
Epoch: 0017   loss = 0.267103
Epoch: 0018   loss = 0.227229
Epoch: 0019   loss = 0.192739
Model Trained.
Accuracy: 0.6295


# CNN with Keras for CIFAR10 Data

In [7]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D, Dense, Flatten, Reshape, Dropout
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [8]:
tf.reset_default_graph()
keras.backend.clear_session()

n_filters=[32,64]

learning_rate=0.001

n_epochs = 10
batch_size = 100

model = Sequential()

model.add(Conv2D(filters=n_filters[0], 
                 kernel_size=4, 
                 padding='same',
                 activation='relu',
                 input_shape=(dataset.height,
                              dataset.width,dataset.depth)
                ) 
         )

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=n_filters[1], 
                 kernel_size=4, 
                 padding='same', 
                 activation='relu'
                ) 
         )

model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(units=1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=dataset.n_classes, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=learning_rate),
              metrics=['accuracy'])

model.fit(dataset.X_train, dataset.Y_train, batch_size=dataset.batch_size,
          epochs=n_epochs)

score = model.evaluate(dataset.X_test, dataset.Y_test)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        1568      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________